In [10]:
import pandas as pd
import numpy as np
import scipy as sp
#import folium
import re
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns
import time

#from google.cloud import translate
#import pycountry
#import emoji

#translate_client = translate.Client()

import sys # for printing process
import unidecode # for normalizing text
from pathlib import Path # check files


#from py_translator import Translator
from google.cloud import translate
translator = translate.Client()



In [15]:
import os
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

/Users/age/keys/ADA-translation-1fb38fd06474.json


In [11]:
translation = translator.translate("hola",target_language='en')
translation['translatedText']

'Hello'

In [12]:
data_folder = './data/'
maps_folder = './maps/'

In [13]:
# CONSTANTS
# unknown values to use
UNKNOWN_NR='-1'
UNKNOWN_STR='unknown'
# delay between translation requests
TRANSLATION_DELAY=0.3
# progress in function
PROGRESS=0



In [17]:
# removes special character, numbers, accents, sets to lower
# case and removes trailing spaces
def normalizeString(string):
    cleaned=''.join([i for i in string if (i.isalnum() & ~i.isdigit()) | i.isspace() ])
    return cleaned.lower().strip()

def showProgress(size):
    global PROGRESS 
    PROGRESS += 1
    progress_value = int(10000*PROGRESS/size)/100
    if (progress_value*100)%1==0:
        sys.stdout.write('\r'+'Progress {}%'.format(progress_value))
        sys.stdout.flush()
def removeDuplcates(array):
    newItems=[]
    for item in array:
        if item and item not in newItems:
            newItems.append(item)
    
    return newItems
# Show NaN percentage of column values in input df    
def showNanPercentage(df,desired_columns):
    for column in desired_columns:
        print("Percentage of NaN in {} is {:.2f}%".format(column,100*len(df[df[column].isna()])/len(df) ))

# Save to file translations dict
def saveTranslations():
    pd.DataFrame.from_dict(translations,orient="index").T.to_csv(data_folder + translationsfile,sep='\t',index=False)


In [35]:
# Translates a value and saves it to translations cache dict
def translateWithCache(value):
    global translations
    # search translated word in translations map
    if value in translations:
        # print("Cached  {} -> {}".format(value,translations[value]))
        return translations[value]
    else:
        try:
            #print(1)
            time.sleep(TRANSLATION_DELAY) 
            #print(2)
            trns_value = translator.translate(value, target_language='en')
            if not trns_value is None:
                new_translation=trns_value['translatedText'].lower()
                #print("Translating {} -> {}".format(value,new_translation))
                translations[value]=new_translation
                return new_translation
            else:
                print("None {} / {} / {}".format(value,type(value), e))
        except Exception as e:
            #err = e
            print("Exception {} / {} / {}".format(value,type(value), e))
        return value

# Save to file a dataframe with gibben name
def saveModuleDF(name,df):
    df.to_csv(data_folder + name + '.csv', sep='\t',index=False)
    print('Saved module: {}'.format(name))


In [19]:
filename = 'en.openfoodfacts.org.products.csv'
countryfile = 'wikipedia-iso-country-codes.csv'
translationsfile = 'translations.csv'
foodfile = 'food.csv'

In [21]:
# cache translations to save translation requests
translations_file = Path(data_folder + translationsfile)
translations = {}

if not translations_file.is_file():
    print('Translations file not found')
else:
    print('Translations file found')
    translations = pd.read_csv(data_folder + translationsfile, 
                               sep='\t',
                               low_memory=False).to_dict("records")[0]
    print('{} translations found'.format(len(translations)))
    
food_file = Path(data_folder + foodfile)
food_df = pd.DataFrame()

if not food_file.is_file():
    print('Food file not found')
    food_df = pd.read_csv(data_folder + filename, 
                      sep='\t',
                      usecols = using_col,
                      quotechar='"', 
                      low_memory=False)
else:
    print('Food file found')
    food_df = pd.read_csv(data_folder + foodfile, 
                      sep='\t',
                      low_memory=False)
    print('{} Food entries found'.format(len(food_df)))


Translations file found
9385 translations found
Food file found
693846 Food entries found


In [22]:
desired_columns=[
    'labels_en',
    'labels',
    'labels_tags'
]
result_column='label_values'
showNanPercentage(food_df,desired_columns)

Percentage of NaN in labels_en is 85.31%
Percentage of NaN in labels is 85.32%
Percentage of NaN in labels_tags is 85.31%


In [23]:
labels_df = food_df.copy()

In [26]:
# formats and translate rows formated as a list of values
def formatAndTranslateRow(row, size, translateNoFormat):
    showProgress(size)

    if type(row) is not list and pd.notnull(row) :
        raw_data = row.lower().split(',')
        data = []
        for value in raw_data:
            # format <langage_code:info>
            if (':') in value:
                info_ = value.split(':')
                if len(info_) == 2:
                    # already in english
                    if info_[0] == 'en':
                        #print("Appending 1 {}".format(info_[1]))
                        data.append(info_[1])
                    # translate to english
                    else:
                        #print("Translating {}".format(info_[1]))
                        data.append(translateWithCache(info_[1]))
                else:
                    data.append(info_)
                    #print("Appending 2 {}".format(info_))
                
            # no format, let's translate it
            else:
                #print("No Cont : {}".format(value))
                if translateNoFormat:
                    data.append(translateWithCache(value))
                else:
                    #print("Appending 3 {}".format(value))
                    data.append(value)
        #print(data)
        return data                       
    else:
        return row
    


In [27]:
labels_nonan_df = labels_df[labels_df[desired_columns[0]].notnull()]

In [28]:
labels_nonan_df[desired_columns[0]]

107                                          Made in France
252                      fr:Viande Française,Made in France
284                                           Contains GMOs
288         Organic,EU Organic,fr:AB Agriculture Biologique
290                                    Kosher,Contains GMOs
293                                               Green Dot
296                                           Contains GMOs
299                                               Green Dot
300                     No artificial flavors,Contains GMOs
325                                             Gluten-free
351                                 Incorrect data on label
358       Not advised for specific people,PDO,Made in Fr...
385       Sustainable farming,UTZ Certified,UTZ Certifie...
410       Low or no sugar,Green Dot,No sugar,With sweete...
411                                           Contains GMOs
422                                           Contains GMOs
424                                     

In [31]:
    PROGRESS=0
    labels_nonan_df[result_column] = labels_nonan_df[desired_columns[0]].apply(
        lambda x: formatAndTranslateRow(x,labels_df.shape[0],False)
    )
    saveTranslations()
    saveModuleDF(result_column,labels_nonan_df)

Progress 14.68%

/anaconda3/envs/ada/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


NameError: name 'saveModuleDF' is not defined

In [36]:
    saveModuleDF(result_column,labels_nonan_df)

Saved module: label_values


In [34]:
len(translations)

15862

In [38]:
labels_nonan_df[labels_nonan_df[result_column].notnull()][result_column]

107                                        [made in france]
252                           [french meat, made in france]
284                                         [contains gmos]
288               [organic, eu organic, ab organic farming]
290                                 [kosher, contains gmos]
293                                             [green dot]
296                                         [contains gmos]
299                                             [green dot]
300                  [no artificial flavors, contains gmos]
325                                           [gluten-free]
351                               [incorrect data on label]
358       [not advised for specific people, pdo, made in...
385       [sustainable farming, utz certified, utz certi...
410       [low or no sugar, green dot, no sugar, with sw...
411                                         [contains gmos]
422                                         [contains gmos]
424                                     

In [48]:
saveTranslations()
labels_df

,product_name,generic_name,quantity,brands,brands_tags,categories,categories_tags,categories_en,manufacturing_places,manufacturing_places_tags,...,proteins_100g,salt_100g,sodium_100g,alcohol_100g,calcium_100g,iron_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.8,1.400,0.551181,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.2,2.040,0.803150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Salade de carottes râpées,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,0.420,0.165354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fromage blanc aux myrtilles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.4,0.250,0.098425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Vainilla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Baguette parisien,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.7,0.678,0.266929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,&quot;Baguette Lyonnais&quot;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.5,0.900,0.354331,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
test=ingredients_df[ingredients_df[result_column].notna()][result_column].loc[65]
test_=test.split(',')
test_

['Rolled oats',
 ' grape concentrate',
 ' expeller pressed canola oil',
 ' sunflower seeds',
 ' almonds',
 ' walnuts oat bran',
 ' sesame seeds',
 ' cashews',
 ' natural vitamin e.']

In [150]:
FOUND_MODULE = result_column in df.columns

True

In [27]:
def removeCommonWords(value):
    words=['long','grain','white','refined','concentrate','natural','dry','roasted','organic','bar','whole','rolled','seasoning','juice','extract']
    value_=value
    for word in words:
        if word in value_:
            value_=value_.replace(word,"")
    return normalizeString(value_)

def cleanIngredients(row):
    showProgress(ingredients_df.shape[0])
    if type(row) is not list and pd.notnull(row) :
        values=row.split(',')
        ingredients=[]
        for item in values:
            ingredient=item
            # format key:value
            # take only value
            if (':') in ingredient:
                info_ = ingredient.split(':')
                if len(info_) == 2:
                    ingredient=info_[1]
            # format value (info)
            # take only values
            if ('(') in ingredient:
                info_ = ingredient.split('(')
                ingredient=info_[0]
                
            # format value [value,...]
            # take all values
            if ('[') in ingredient:
                info_ = ingredient.split('[')
                if len(info_) >= 2:
                    ingredients.append(removeCommonWords(normalizeString(info_[0])))
                    ingredient=info_[1]
            # format value [value,...]
            # take all values
            if (' or ') in ingredient:
                info_ = ingredient.split('or')
                if len(info_) >= 2:
                    ingredients.append(removeCommonWords(normalizeString(info_[0])))
                    ingredient=info_[1]
            # format value - info
            # take only values
            if (' - ') in ingredient:
                info_ = ingredient.split('-')
                ingredient=info_[0]
                
            # avoid empty strings and removeCommonWords
            ingredient=removeCommonWords(normalizeString(ingredient))
            if ingredient:
                ingredients.append(ingredient)
        
        # remove duplicate elements
        ingredients_=removeDuplcates(ingredients)
        return ingredients_
    else:
        return row

In [169]:
PROGRESS=0
cleanIngredients(test)

0.01
Progress 1.0%

['oats',
 'grape',
 'expeller pressed canola oil',
 'sunflower seeds',
 'almonds',
 'walnuts oat bran',
 'sesame seeds',
 'cashews',
 'vitamin e']

In [113]:
test

'Organic rolled oats, flame raisins, organic coconut chips, organic maple syrup, sunflower oil, walnuts, roasted diced almonds, wheat germ, honey, crunchy almond butter (dry roasted almonds), hazelnuts, almonds, organic ground cinnamon, organic vanilla extract.'

In [189]:
PROGRESS=0
ingredients_df[result_column] = ingredients_df[result_column].apply(
    lambda x: cleanIngredients(x)
)

Progress 100.0%

10    [erythorbate de sodium, caramel, tomate, mayon...
15    [lait entier, sucre, amidon de mais, cacao, ag...
22    [baguette poite vin pain baguette, fqrine de b...
31    [paln suedois, farine de ble, eau, farine de s...
33    [taboule, legumes, huile de colza, sel, menthe...
34    [bananas, vegetable oil, corn oil andor palm o...
35    [peanuts, wheat flour, sugar, rice flour, tapi...
39    [fromage blanc  mg, creme de marron vanillee, ...
44    [pain aux  cereqles, farine, gluten de ble, eu...
46    [baguette bressan pain baguette, fqrine de ble...
63    [hazelnuts, cashews, walnuts almonds, sunflowe...
64                                            [polenta]
65    [oats, grape, expeller pressed canola oil, sun...
66                                               [rice]
67    [org oats, org hemp granola, evaporated cane, ...
68    [chocolate liquor, raw cane sugar, cocoa butte...
69         [expeller pressed, high oleic sunflower oil]
70                                       [adzuki